# 6장 : Profile Page and Avatars

## 사용자 프로필 페이지

- app/routes.py 에 '/user/\<username\>' url 매핑을 추가함. dynamic 하게 url을 만듦.
- .first_or_404() : query에 포함된 메소드. 결과가 있으면 첫번째 것을 주고, 없으면 자동으로 404 error를 클라이언트한테 줌.  
※ .first() : 결과가 없으면 None을 리턴함.

- app/templates/user.html 추가하고, base.html에도 Profile 메뉴 추가.
- url_for('user', username=current_user.username) : url 파라미터 넣기

## 아바타

- 서버에 이미지들을 대량으로 모으는 것이 아니라, Gravatar 서비스로 모든 사용자에게 이미지를 제공함.
- https://www.gravatar.com/avatar/\<사용자 email의 MD5 해쉬값\> : 해당 url로 들어가면 80x80 픽셀의 이미지를 볼 수 있음.
※ '?s=128' 추가하면 128x128 이미지, 'd=identicon' 이미지가 등록되어있지 않은 사용자들에게 그래픽 이미지를 줌.
- app/models.py 의 User 클래스에 avatar함수를 추가함
- app/templates/user.html 에 이미지를 나타낼 부분을 추가함(이름 옆에는 큰 이미지 user.avatar(128), post 옆에는 작은 이미지)

## Jinja2 Sub-Templates 사용하기

- sub-template이 필요한 이유 : user.html의 post 형태로, index.html에도 같이 표시하고 싶음. 그런데 copy & paste 하면, 나중에 수정이 일어났을 때 모두 수정해야한다는 걸 기억해야함.
- app/templates/_post.html 을 추가하여 하나의 post를 렌더링할 수 있는 서브 템플릿을 만듦. 언더바로 서브 템플릿임을 나타냄.
- **{% include '_post.html' %}** : 서브 템플릿 삽입

## About me, last seen 추가

- app/models.py User 클래스에 about_me, last_seen 컬럼추가
- flask db migrate > flask db upgrade로 SQLite DB에 반영.(기존 데이터들이 유지된 채로 수행됨)
- app/templates/user.html 에 두 필드를 표시할 수 있는 란 추가

## 사용자의 Last Visit 시간 기록하기

- 사용자가 서버에 요청을 보낼 때마다 last_seen 필드를 현재 시간으로 쓰고 싶음
- **@app.before_request** 데코레이터 : view function이 실행되기 직전에 수행할 함수를 등록함.
- routes.py 에 @app.before_request 등록함. current_user를 부르기 위해서 Flask-Login이 user loader 함수를 부르고, 이 함수 안에서 타겟 사용자를 db세션에 올림.(그래서 세션에 이미 있으므로 session.add()하지 않아도 됨)
※ db가 위치에 영향을 받지 않기 위해서 UTC time zone을 사용함

## 프로필 편집기

- app/forms.py 에 EditProfileForm 클래스를 추가함. (TextAreaField)
- app/templates/edit_profile.html 파일 추가  
Username을 비워서 invalid form을 만들면 '이 입력란을 작성하세요'라는 툴팁이 뜸
- app/routes.py 에 '/edit_profile' url 매핑 추가  
db에서 user를 직접 select하는 것이 아니라, current_user 로 user를 load해서 세션에 올린 다음 수정함.
- app/templates/user.html 에서 'Edit your profile' 메뉴 추가  
사용자가 본인일 경우에만.(user == current_user)
- Q. html에서 form 태그 안에 action="" 을 해주는데, 어떻게 해당 url로 가는가?  
안 써주면 default value이고, action의 default value는 submit 하는 페이지의 url임. 예를 들어 http://127.0.0.1:5000/register 이 register.html의 url인데, 여기서 submit하면 똑같이  http://127.0.0.1:5000/register 로 들어감. 다만 method를 post로 지정해주었기 때문에 post 방식으로 들어감.